# 20기 KNN 정규세션 과제

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [27]:
import pandas as pd
df = pd.read_csv("blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA

In [28]:
print(df.shape)
print(pd.DataFrame(df.info()))

(4998, 12)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB
Empty DataFrame
Colu

In [29]:
print(df.isnull().sum())
df.isnull().sum()/df.isnull().count()*100

User_ID                          0
Product_ID                       0
Gender                           0
Age                              0
Occupation                       0
City_Category                    0
Stay_In_Current_City_Years       0
Marital_Status                   0
Product_Category_1               0
Product_Category_2            1533
Product_Category_3            3454
Purchase                         0
dtype: int64


User_ID                        0.000000
Product_ID                     0.000000
Gender                         0.000000
Age                            0.000000
Occupation                     0.000000
City_Category                  0.000000
Stay_In_Current_City_Years     0.000000
Marital_Status                 0.000000
Product_Category_1             0.000000
Product_Category_2            30.672269
Product_Category_3            69.107643
Purchase                       0.000000
dtype: float64

In [30]:
#결측치가 69%인 product_category_2 드랍
df = df.drop('Product_Category_3', axis=1)

df = df.drop('User_ID', axis=1)
df = df.drop('Product_ID', axis=1)

In [31]:
from sklearn.impute import SimpleImputer
import numpy as np
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer = imputer.fit(pd.DataFrame(df['Product_Category_2']))


df['Product_Category_2'] = imputer.transform(pd.DataFrame(df['Product_Category_2']))

In [32]:
imputer = imputer.fit(pd.DataFrame(df['Marital_Status']))
df['Marital_Status'] = imputer.transform(pd.DataFrame(df['Marital_Status']))

In [33]:
df.isnull().sum()/df.isnull().count()*100

Gender                        0.0
Age                           0.0
Occupation                    0.0
City_Category                 0.0
Stay_In_Current_City_Years    0.0
Marital_Status                0.0
Product_Category_1            0.0
Product_Category_2            0.0
Purchase                      0.0
dtype: float64

In [34]:
print(df['Gender'].value_counts())
print(df['Age'].value_counts())
print(df['City_Category'].value_counts())

M    3765
F    1233
Name: Gender, dtype: int64
26-35    1996
36-45    1000
18-25     906
46-50     414
51-55     350
55+       195
0-17      137
Name: Age, dtype: int64
B    2102
C    1555
A    1341
Name: City_Category, dtype: int64


In [35]:
#gender을 binary값으로 바꿈
df.loc[:, 'Gender'] = np.where(df['Gender'] == 'M', 1, 0)

df = df.rename(columns={
                'Product_Category_1': 'Category1',
                'Product_Category_2': 'Category2',
                'City_Category': 'City',
                'Stay_In_Current_City_Years': 'City_Stay'
})

<ipython-input-35-4271e5fc1015>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:, 'Gender'] = np.where(df['Gender'] == 'M', 1, 0)


In [36]:
from sklearn.preprocessing import  OneHotEncoder
cats = ['Occupation', 'Age', 'City', 'Category1','Category2','City_Stay']

#creating the encoder, fit it to our data
encoder = OneHotEncoder().fit(df[cats])

In [37]:
encoder.get_feature_names_out(cats)

array(['Occupation_0', 'Occupation_1', 'Occupation_2', 'Occupation_3',
       'Occupation_4', 'Occupation_5', 'Occupation_6', 'Occupation_7',
       'Occupation_8', 'Occupation_9', 'Occupation_10', 'Occupation_11',
       'Occupation_12', 'Occupation_13', 'Occupation_14', 'Occupation_15',
       'Occupation_16', 'Occupation_17', 'Occupation_18', 'Occupation_19',
       'Occupation_20', 'Age_0-17', 'Age_18-25', 'Age_26-35', 'Age_36-45',
       'Age_46-50', 'Age_51-55', 'Age_55+', 'City_A', 'City_B', 'City_C',
       'Category1_1', 'Category1_2', 'Category1_3', 'Category1_4',
       'Category1_5', 'Category1_6', 'Category1_7', 'Category1_8',
       'Category1_9', 'Category1_10', 'Category1_11', 'Category1_12',
       'Category1_13', 'Category1_14', 'Category1_15', 'Category1_16',
       'Category1_17', 'Category1_18', 'Category1_19', 'Category1_20',
       'Category2_2.0', 'Category2_3.0', 'Category2_4.0', 'Category2_5.0',
       'Category2_6.0', 'Category2_7.0', 'Category2_8.0', 'Catego

In [38]:
endcoded_data = pd.DataFrame(encoder.transform(df[cats]).toarray(),index=df.index, columns=encoder.get_feature_names_out(cats))
endcoded_data.head()

,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,Occupation_6,Occupation_7,Occupation_8,Occupation_9,...,Category2_14.0,Category2_15.0,Category2_16.0,Category2_17.0,Category2_18.0,City_Stay_0,City_Stay_1,City_Stay_2,City_Stay_3,City_Stay_4+
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [39]:
df = pd.concat([df, endcoded_data],sort=False,axis=1)

df=df.drop(cats, axis=1)

In [41]:
df = df.fillna(0)
df.head(15)

,Gender,Marital_Status,Purchase,Occupation_0,Occupation_1,Occupation_2,Occupation_3,Occupation_4,Occupation_5,Occupation_6,...,Category2_14.0,Category2_15.0,Category2_16.0,Category2_17.0,Category2_18.0,City_Stay_0,City_Stay_1,City_Stay_2,City_Stay_3,City_Stay_4+
0,0,0,2010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0,4483,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,7696,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0,0,16429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0,0,5780,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0,0,9821,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6,0,0,12707,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7,0,0,7108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0,0,16521,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9,0,0,1886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [42]:
X = df.drop('Gender',axis=1)
y = df.pop('Gender')

In [43]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

## 2. KNN 구현 & 파라미터 튜닝

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size= 0.25)

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train,y_train)

KNeighborsClassifier()

In [46]:
knn.score(X_test,y_test)

0.736

In [47]:
parameters = { 'algorithm' : ['ball_tree', 'kd_tree', 'brute'],
               'leaf_size' : [18,25,27,32,34],
               'n_neighbors' : [3,7,10,12,13]
              }

from sklearn.model_selection import GridSearchCV
gridsearch = GridSearchCV(knn, parameters,verbose=3)
gridsearch.fit(X_train,y_train)
gridsearch.best_params_

Fitting 5 folds for each of 75 candidates, totalling 375 fits
[CV 1/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3;, score=0.723 total time=   0.6s
[CV 2/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3;, score=0.737 total time=   0.6s
[CV 3/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3;, score=0.713 total time=   0.6s
[CV 4/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3;, score=0.720 total time=   0.6s
[CV 5/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=3;, score=0.717 total time=   0.6s
[CV 1/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7;, score=0.755 total time=   0.6s
[CV 2/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7;, score=0.737 total time=   0.6s
[CV 3/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7;, score=0.728 total time=   0.6s
[CV 4/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7;, score=0.741 total time=   0.6s
[CV 5/5] END algorithm=ball_tree, leaf_size=18, n_neighbors=7;, score=0.732 total t

{'algorithm': 'ball_tree', 'leaf_size': 18, 'n_neighbors': 13}

## 3. Evaluation

In [48]:
knn = KNeighborsClassifier(algorithm = 'auto', leaf_size =35, n_neighbors =5)
knn.fit(X_train,y_train)
knn.score(X_test,y_test)

0.736